<a href="https://colab.research.google.com/github/LangqingZou/Math-509/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns
import pylab as pl
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier as DNN

In [86]:
X = pd.read_csv('drive/MyDrive/Math_509_Project/data_x.csv')
Y = pd.read_csv('drive/MyDrive/Math_509_Project/data_y.csv')

In [29]:
# standarlized the dataset
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

In [78]:
# split into training and testing datasets
x_train, x_test, y_train, y_test = train_test_split(X, Y.astype('float'), test_size=0.20, random_state=53)
# y_train = y_train.str.strip()
# y_test = y_test.astype(str).strip().to_numpy()


In [85]:
# c = y_train.to_numpy()
# c.dtype
x_train.dtype

dtype('float64')

# Linear Regression Model

In [50]:
## construct linear regression model
clf = LinearRegression().fit(x_train,y_train)
 
## predicted values
y_pred = clf.predict(x_test)

In [51]:
print("coefficients =",clf.coef_)    # coefficients
print("intercept b =",clf.intercept_) # intercept
print('Mean Absolute Error =', mean_absolute_error(y_test, y_pred))
print('Mean Squared Error =', mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error =', np.sqrt(mean_squared_error(y_test, y_pred)))
print('R2 Score:', r2_score(y_test, y_pred))

coefficients = [[ 4.21597868e+10 -8.15596162e+09  1.01650380e+11  9.22127068e-03
   5.03935814e-02  2.04100609e-02 -6.71666861e-02  4.41866517e-02
  -1.56073213e-01 -4.21597868e+10  3.70004820e+09  7.06789227e+09
   7.25907645e+09  5.52396430e+09 -3.80910208e+10 -9.57959917e+10]]
intercept b = [0.02642093]
Mean Absolute Error = 0.7547827352931948
Mean Squared Error = 0.9098722652454524
Root Mean Squared Error = 0.9538722478641741
R2 Score: 0.07402693880586642


# Lasso Regression Model

In [52]:
lasso=Lasso(alpha=0.01,max_iter=10000)
lasso.fit(x_train,y_train)
print("Lasso training dataset score："+str(r2_score(y_train,lasso.predict(x_train))))
print("Lasso testing dataset score："+str(r2_score(y_test,lasso.predict(x_test))))

Lasso training dataset score：0.07125537310673902
Lasso testing dataset score：0.07264416855164846


# Ridge Regression Model

In [53]:
ridge = Ridge(alpha=1.0)
ridge.fit(x_train, y_train)
print("Ridge training dataset score："+str(r2_score(y_train,ridge.predict(x_train))))
print("Ridge testing dataset score："+str(r2_score(y_test,ridge.predict(x_test))))

Ridge training dataset score：0.0723039669316472
Ridge testing dataset score：0.07402759045455787


# Binary Tree

# Neural Network

In [104]:
y_train = y_train.apply(lambda x: x.str.replace(' ',''))
# dataFrame[headers] = dataFrame[headers].apply(lambda x: x.str.replace(',',''))

AttributeError: ignored

In [91]:
# hidden_layer_sizes=(2, 1) hidden层2层,第一层2个神经元，第二层1个神经元)，2层隐藏层，也就有3层神经网络
nn = DNN(solver='sgd', alpha=0.01,hidden_layer_sizes=(2, 1), random_state=1)
nn.fit(x_train, y_train)
print("Neural networking training dataset score："+str(r2_score(y_train,ridge.predict(x_train))))
print("Neural networking testing dataset score："+str(r2_score(y_test,ridge.predict(x_test))))

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


ValueError: ignored